In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pixstem.api as ps
from PIL import Image
import hyperspy.api as hs
import csv
#%matplotlib qt

file = hs.load("input.blo")

In [2]:
def distance(x1, y1, x2, y2):
    return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))

def pos_dist(x1, y1, x2, y2):
    if (x2 > x1 and y2 > y1):
        return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
    else:
        return -1.0
def findCenter(im, peak):
    center = (0,0)
    maximum = 0
    for (x,y) in np.ndenumerate(peak):
        for (a, b) in y:
            if (int(a) < len(im) and int(b) < len(im) and im[int(a)][int(b)] > maximum):
                maximum = im[int(a)][int(b)]
                center = (b, a)
    return center

In [ ]:
distances = []
writer = csv.writer(open("outputDistances.csv", "w")) # opens csv writer to write to input file
for i in range(300):
    temp = []
    for j in range(500):
        print(i, j)
        s = ps.PixelatedSTEM(hs.signals.Signal2D(file.inav[i, j]))
        imarray = np.array(s)
        s = s.rotate_diffraction(0,show_progressbar=False)
        ############################################################################################################################
        st = s.template_match_disk(disk_r=5, lazy_result=False, show_progressbar=False)
        peak_array = st.find_peaks(lazy_result=False, show_progressbar=False)
        peak_array_com = s.peak_position_refinement_com(peak_array, lazy_result=False, show_progressbar=False)
        s_rem = s.subtract_diffraction_background(lazy_result=False, show_progressbar=False)
        peak_array_rem_com = s_rem.peak_position_refinement_com(peak_array, lazy_result=False, show_progressbar=False)
        ############################################################################################################################
        center = findCenter(imarray, peak_array_rem_com)

        # finds the specific spot and adding that distance to the array
        posDistance = 0
       
        for (x,y) in np.ndenumerate(peak_array_rem_com):
            prev = (0, 0)
            for (a, b) in y:
                if abs(center[0] - b) < 1E-5 and abs(center[1] - a) < 1E-5:
                    posDistance = distance(center[0], center[1], prev[1], prev[0])
                    break
                prev = (a, b)
                #is = pos_dist(center[0], center[1], b, a)
                #if not dis < 0.00000001 and dis < posDistance:
                    #posDistance = dis
                #posDistance = distance(center[0], center[1], b, a)
        temp.append(round(posDistance, 2))
        distances.append(round(posDistance, 2))
    writer.writerow(temp) # writes the values in the temp array to a row of the csv file

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


In [ ]:
distances.sort()
#print(distances)
x_pos = np.arange(np.min(distances), np.max(distances), 0.01) # this 0.01 is the distance between each x-axis label. So for example it goes 1.0,1.01,1.02,1.03...
x_pos = [round(num, 2) for num in x_pos]
y_pos = np.arange(len(x_pos))
counts = [] # has the count of each value in the same order as x_pos
for i in x_pos:
    ctr = 0
    for x in distances:
        if i <= x < i + 0.01: # change the 0.01 to whatever number you use above in the first comment
            ctr += 1
    counts.append(ctr)

plt.bar(y_pos, counts, align='center', alpha=0.95) # creates the bar plot
plt.xticks(y_pos, x_pos, fontsize = 5)
plt.xlabel('Distance from center peek', fontsize = 5)
plt.ylabel('Counts', fontsize = 5)
plt.title('Distance Counts', fontsize = 5)

ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=90)
ax.tick_params(axis='x', which='major', labelsize=5)
ax.tick_params(axis='y', which='major', labelsize=5)

[l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0] 
# The '2' is the every nth number of labels its shows on the x-axis. So rn is shows every 2nd label. 

plt.gcf().subplots_adjust(bottom = 0.23)
plt.rcParams["figure.dpi"] = 500
plt.show()